///---
layout: post
title: "Numba를 이용한 Cuda 프로그램 예제"
comments: true
share: true
date: 2019-07-07 12:58:00
description: Python에서 Numba를 이용한 Cuda 프로그래밍 예제를 소개한다.
tags: python cuda
sitemap :
  changefreq : daily
  priority : 1.0
///---

# Numba를 이용한 Cuda 프로그램 예제

## Sum

### Sum Kernel Code

In [1]:
from numba import cuda
import numpy as np

@cuda.jit
def sum_kernel(arr):
    pass

### Sum Host code

In [23]:
import numpy as np
from numpy.random_intel import standard_normal

n_sample = 2 ** 28

array = standard_normal(n_sample)

sum_cpu = np.sum(array)

print(sum_cpu)

7535.841265250778
